<div style="float:right; color:red; font-weight:bold;">Rename this file before you work on it!</div>
# Pandas Spark Exercise 2

We use the [Adventure Works dataset](https://msdn.microsoft.com/en-us/library/hh403424.aspxto) to compare queries
in SQL and Spark.

## Task
1. Fill in the Python-Spark expressions to produce a table as described in the problem statements.
2. The SQL expression may give you a hint. It also allows you to see both systems side-by-side.
3. Documentation at http://spark.apache.org/docs/latest/sql-programming-guide.html

# Setup

## Spark
This example runs with a Jupyter kernel that launches a Spark Context by default. If this is not the case on your system you have to initialize the Spark environment first.

In [ ]:
#"""
#Load packages and create context objects...
#"""
import os
import platform
import sys
import pandas

if not 'sc' in vars():
    print """
***********************************************
*** Warning: Spark needs to be initialized ****
***********************************************
    """
else:
    print("""Already running
          ____              __
         / __/__  ___ _____/ /__
        _\ \/ _ \/ _ `/ __/  '_/
       /__ / .__/\_,_/_/ /_/\_\   version %s
          /_/
    """ % sc.version)

if not 'sqlCtx' in vars():
    sqlCtx = SQLContext(sc)
print 'Spark Context available as `sc`'
print 'Spark SQL Context (%s) available as `sqlCtx`'%str(type(sqlCtx))
### The spark context doesn't really know the URL to the job manager... this would be different on every system
print "Monitor this application at http://data-science-02.atl.primedia.com:8088/proxy/"+sc.applicationId



## Additional Spark Packages for Python

In [ ]:
import pyspark.sql.functions as F

## Load Tables

In [ ]:
# %load adventureworks_spark.py
if not 'sqlCtx' in vars():
   sqlCtx = SQLContext(sc)

Employees_df = sqlCtx.read.format('com.databricks.spark.csv')\
    .options(header=True, inferschema=True)\
    .load('/user/pmolnar/data/AdventureWorks/Employees.csv.gz')

Territory_df = sqlCtx.read.format('com.databricks.spark.csv')\
    .options(header=True, inferschema=True)\
    .load('/user/pmolnar/data/AdventureWorks/SalesTerritory.csv.gz')

Orders_df = sqlCtx.read.format('com.databricks.spark.csv')\
    .options(header=True, inferschema=True)\
    .load('/user/pmolnar/data/AdventureWorks/ItemsOrdered.csv.gz')

Customers_df = sqlCtx.read.format('com.databricks.spark.csv')\
    .options(header=True, inferschema=True)\
    .load('/user/pmolnar/data/AdventureWorks/Customer.csv.gz')



# Queries

### 1a. Show me all the employees, and if any are salespeople then show me the details about their sales territory

In [ ]:
Employees_df.join(Territory_df, on='TerritoryID', how='left').limit(20).toPandas()

### 1b. Change the columns above
From Employees table use: EmployeeID, FirstName, LastName, TerritoryID
From the SalesTerritory table use: show all the columns except for the TerritoryID

In [ ]:
Employees_df.withColumn('EmployeeName', F.concat_ws('FirstName', 'LastName'))\
    .join(Territory_df, on='TerritoryID', how='left')\
    .select(['EmployeeID', 'EmployeeName', 'TerritoryID', 'Name', 'CountryCode', 'Region', 'SalesYTD', 'SalesLastYear'])\
    .limit(20).toPandas()

### 1c. For the list above, limit the results to just salespeople

In [ ]:
Employees_df.withColumn('EmployeeName', F.concat_ws('FirstName', 'LastName'))\
    .join(Territory_df, on='TerritoryID', how='inner')\
    .select(['EmployeeID', 'EmployeeName', 'TerritoryID', 'Name', 'CountryCode', 'Region', 'SalesYTD', 'SalesLastYear'])\
    .limit(20).toPandas()

### 2a. Give me a list of customers

### 2b. For those customers, also tell me which sales territory they fall in

### 3a. Give me a list of sales territories

### 3b. For those sales territories, also show what customers fall under them

### 3c. Make sure, if you didn't already, that any sales territories with 0 customers are also shown in the list

### 4a. Give me a list of the customers we have in North Carolina, and note how many there are.

	
### 4b. For our North Carolina customers, show me the items they have ordered

### 4c. Change it so we're only looking at customers who are in Arizona

### 4d. For each of the items ordered, show the total price of the order (sometimes they ordered more than 1 item)

### 4e. For the Arizona customers above, show the maximum total price per customer. Choose the columns wisely

### 4f. For the results above, only show the ones where the MaxTotalPrice is more than $5.00

### BONUS 1: Take the query from #3c and add a column called "CityRegion" that combines dbo.Customers.City and 

### BONUS 2: Fix the problem above where CityRegion is NULL sometimes; change it to, for example, "n/a - North America"

### OVERACHIEVER 1: Why do you think a database is designed so the data is split up into different tables?  Why not just put it all in one table?

### OVERACHIEVER 2a: Take the query from 1b and sort it by the sales region so that all the salespeople are the top of the list

### OVERACHIEVER 2b: From the query above, if the sales territory is outside Europe, do not display it (but we still want to see all the employees in the company!)